In [ ]:
3.1 Load More Example (Hacker News “More” Link)

In [3]:
import asyncio
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig,CacheMode,BrowserConfig

async def main():
    base_browser = BrowserConfig(
        browser_type="chromium",
        headless=False
    )
    #Step 1: Load initial Hacker News page
    
    config = CrawlerRunConfig(
        wait_for="css:.athing:nth-child(30)",  # Wait for 30 items
        session_id="hn_session",
        cache_mode=CacheMode.BYPASS
    )
    async with AsyncWebCrawler(config=base_browser) as crawler:
        result = await crawler.arun(
            url="https://news.ycombinator.com",
            config=config
        )
        print("Initial items loaded.")

        # Step 2: Let's scroll and click the "More" link
        load_more_js = [
            "window.scrollTo(0, document.body.scrollHeight);",
            # The "More" link at page bottom
            "document.querySelector('a.morelink')?.click();"  
        ]

        next_page_conf = CrawlerRunConfig(
            js_code=load_more_js,
            wait_for="""js:() => {
                return document.querySelectorAll('.athing').length > 30;
            }""",
            # Mark that we do not re-navigate, but run JS in the same session:            
            cache_mode=CacheMode.BYPASS
        )

        # Re-use the same crawler session
        result2 = await crawler.arun(
            url="https://news.ycombinator.com",  # same URL but continuing session
            config=next_page_conf
        )
        total_items = result2.cleaned_html.count("athing")
        print("Items after load-more:", total_items)

if __name__ == "__main__":
    await main()


[INIT].... → Crawl4AI 0.6.3 

TargetClosedError: BrowserContext.close: Target page, context or browser has been closed